In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import pydicom

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader, random_split

import numpy as np

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2


In [ ]:
train_path = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images"
test_path = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images"


In [ ]:
train_csv_path="/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv"
df_target = pd.read_csv(train_csv_path)

# Afficher les premières lignes du DataFrame pour vérifier le contenu
df_target[:686]

In [ ]:
train_label_csv_path="/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv"
df_label = pd.read_csv(train_label_csv_path)

# Afficher les premières lignes du DataFrame pour vérifier le contenu
df_label[df_label["study_id"]=="44036939"]

In [ ]:
train_series_csv_path="/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv"
df_series = pd.read_csv(train_series_csv_path)

# Afficher les premières lignes du DataFrame pour vérifier le contenu
df_series.head()

In [ ]:
submission_csv_path="/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/sample_submission.csv"
df = pd.read_csv(submission_csv_path)

# Afficher les premières lignes du DataFrame pour vérifier le contenu
df.head()

In [ ]:
filter_df_label=df_label[(df_label["study_id"]==1524089207) & (df_label["series_id"]==2221300812) & (df_label["instance_number"]==12)]
for index, row in filter_df_label.iterrows():
    print(f"Study ID: {row['study_id']}, Series ID: {row['series_id']}")
    file_name=row['condition'].replace(" ","_")+"_"+row['level'].replace("/","_")
    print(f"name:{file_name}")
    tmp_df=df_target[["study_id",file_name.lower()]]
    tmp_df=tmp_df[tmp_df["study_id"]==1524089207]
    for index, row in tmp_df.iterrows():
        print(row[file_name.lower()])
    print("\n")
tmp_df

In [ ]:
dcm_file_path = "/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/100206310/1012284084/8.dcm"

# Lire le fichier DICOM
dicom_data = pydicom.dcmread(dcm_file_path)

# Extraire l'image du fichier DICOM
image = dicom_data.pixel_array
image = image.astype(np.float32)
print(image.shape)
image = np.squeeze(image)
plt.imshow(image, cmap='gray')
plt.title('DICOM Image')
plt.axis('off')
plt.show()

In [ ]:
condition_dict={col:i for i, col in enumerate(df_target.columns[1:])}
print(condition_dict)
condition_array=[col for col in df_target.columns[1:]]
print(condition_array)

In [ ]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import math

def str_is_nan(s):
    try:
        num = float(s)
    except ValueError:
        return False
    return math.isnan(num)

class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.files = []
        self.labels = []
        self.condition_labels = []
        self.names = []
        self.label_to_index = { 'Normal_Mild': 0, 'Moderate' : 1,    'Severe' : 2}
        self.index_to_label = ['Normal_Mild', 'Moderate',    'Severe' ] 
        self.condition_labels_to_index = condition_dict
        self.index_to_condition_labels = condition_array
        self.failed_count=0
        study_tqdm = tqdm(os.listdir(train_path), desc=f'Creation du dataset...', leave=True)
        for study_id in study_tqdm:
            study_folder_path = os.path.join(train_path, study_id)
            
            if os.path.isdir(study_folder_path):
                # Parcourir chaque dossier de série dans le dossier study_id
                for series_id in os.listdir(study_folder_path):
                    series_folder_path = os.path.join(study_folder_path, series_id)
                    
                    if os.path.isdir(series_folder_path):
                        # Compter le nombre de fichiers .dcm dans chaque dossier de série
                        for f in os.listdir(series_folder_path):
                            if f.endswith('.dcm'):
                                filter_df_label=df_label[(df_label["study_id"]==int(study_id)) & (df_label["series_id"]==int(series_id)) & (df_label["instance_number"]==int(f.replace(".dcm","")))]
                                for index, row in filter_df_label.iterrows():
                                    file_name=row['condition'].replace(" ","_")+"_"+row['level'].replace("/","_")
                                    file_name=file_name.lower()
                                    tmp_df=df_target[["study_id",file_name]]
                                    tmp_df=tmp_df[tmp_df["study_id"]==int(study_id)]
                                    for index, row in tmp_df.iterrows():
                                        key=row[file_name]
                                        if not str_is_nan(key) :
                                            self.names.append(study_id+"_"+file_name)
                                            self.condition_labels.append(self.condition_labels_to_index[file_name])
                                            self.files.append(os.path.join(series_folder_path, f)) 
                                            self.labels.append(self.label_to_index[key.replace("/","_")])
                                        else:
                                            self.failed_count+=1
        if self.failed_count!=0: print(f"There was {self.failed_count} failures")           
        self.transform = transform

        if len(self.files) == 0:
            raise ValueError("No image (dcm) files found in the specified directory.")
#
    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        dicom_data = pydicom.dcmread(dcm_file_path)

        image = dicom_data.pixel_array
        image = image.astype(np.float32)
        
        if self.transform:
            image = self.transform(image)
        
        label = self.labels[idx]
        condition = self.condition_labels[idx]
        name=self.names[idx]
        return image, condition,label,name

In [ ]:
dataset = ImageDataset(train_path, transform=None)


In [ ]:
# la classe ImageFolder assigne automatiquement un label pour chaque nom de classe (class -> idx)
print('class -> idx : ',dataset.label_to_index)

# on aura besoin d'un dictionnaire qui fait le sens inverse (idx -> class)
idx_to_class = {dataset.label_to_index[class_name]: class_name for class_name in  dataset.label_to_index}
print('idx -> class : ',idx_to_class)
print(len(idx_to_class))

print('class -> idx : ',dataset.condition_labels_to_index)

# on aura besoin d'un dictionnaire qui fait le sens inverse (idx -> class)
idx_to_class = {dataset.condition_labels_to_index[class_name]: class_name for class_name in  dataset.condition_labels_to_index}
print('idx -> class : ',idx_to_class)
print(len(idx_to_class))

In [ ]:
import torch
from torch.utils.data import DataLoader

def get_mean_and_std(dataset):
    """
    Calcule la moyenne et l'écart type des canaux de l'ensemble de données d'images à une dimension.

    Args:
    dataset (torch.utils.data.Dataset): Dataset pour calculer la moyenne et l'écart type.

    Returns:
    tuple: Moyenne et écart type des canaux de l'ensemble de données.
    """
    loader = DataLoader(dataset, batch_size=1,num_workers=4, shuffle=False)
    
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    loader_tqdm = tqdm(loader, desc=f'Calcul des params de normalisation...', leave=True)
    for data in loader_tqdm :
        inputs, c,_,n = data  # Extraction des données et étiquettes
        inputs = inputs.squeeze()  # Suppression de la dimension du lot
        channels_sum += torch.mean(inputs, dim=0)
        channels_squared_sum += torch.mean(inputs**2, dim=0)
        num_batches += 1
    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean**2)**0.5
    return mean, std

In [ ]:
mean,std = get_mean_and_std(dataset)
mean, std

In [ ]:
std[std == 0] = 1e-8 
mean[mean == 0] = 1e-8 

In [ ]:
# Division des indices pour l'entraînement, la validation et le test
np.random.seed(42)
indices = np.arange(len(dataset))
np.random.shuffle(indices)
num_train = int(0.7 * len(indices))
num_val = int(0.2 * len(indices))

train_indices = indices[:num_train]
val_indices = indices[num_train:num_train + num_val]
test_indices = indices[num_train + num_val:]

In [ ]:
from torch.utils.data import Dataset

class TransformingSubset(Dataset):
    def __init__(self, subset, transform=None):
        """
        subset: Un objet Subset de PyTorch contenant les indices et le dataset original.
        transform: Les transformations à appliquer aux éléments du dataset.
        """
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        # Récupérer l'élément depuis le subset
        x, y,z,w = self.subset[index]

        # Appliquer les transformations
        if self.transform:
            x = self.transform(x)

        return x, y,z,w

    def __len__(self):
        return len(self.subset)


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from albumentations.core.composition import BboxParams, KeypointParams
class AlbumentationsAdapter:
    def __init__(self, transform):
        self.transform = transform

    def __call__(self, img):
        img = np.array(img)
        return self.transform(image=img)['image']

In [ ]:
class Cutout(object):
    def __init__(self, mask_size, p=0.5):
        self.mask_size = mask_size
        self.p = p

    def __call__(self, img):
        if np.random.rand() > self.p:
            return img

        h, w = img.size()[1], img.size()[2]
        mask_size_half = self.mask_size // 2

        cxmin, cxmax = mask_size_half, w - mask_size_half
        cymin, cymax = mask_size_half, h - mask_size_half

        if cxmin >= cxmax or cymin >= cymax:
            return img
        
        cx = np.random.randint(cxmin, cxmax)
        cy = np.random.randint(cymin, cymax)

        xmin, xmax = cx - mask_size_half, cx + mask_size_half
        ymin, ymax = cy - mask_size_half, cy + mask_size_half

        img[:, ymin:ymax, xmin:xmax] = 0
        return img

In [ ]:

train_transform_v2 = AlbumentationsAdapter(A.Compose([
    # Assurez-vous que les dimensions sont appropriées pour vos spectrogrammes si nécessaire
    #transforms.ToPILImage(), 
    transforms.Grayscale(num_output_channels=3),
    ToTensorV2(),
    #A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    #A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
    
    
]))

train_transform_v3 = AlbumentationsAdapter(A.Compose([
    # Assurez-vous que les dimensions sont appropriées pour vos spectrogrammes si nécessaire
    #transforms.ToPILImage(), 
    transforms.Grayscale(num_output_channels=3),
    ToTensorV2(),
    #A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=40, p=0.5),
    A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
    A.GridDistortion(p=0.5),
    A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=0.5),
    A.GaussianBlur(blur_limit=7, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.ChannelShuffle(p=0.5),
    A.CLAHE(clip_limit=2, p=0.5),
    A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.3, brightness_coeff=2.5, p=0.5),
    A.RandomRain(drop_length=5, drop_width=1, rain_type='drizzle', p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0),
    
]))

train_transform = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Grayscale(num_output_channels=3),
   
    transforms.ToTensor(), 
    transforms.Normalize(mean.unsqueeze(0), std.unsqueeze(0)),
     Cutout(mask_size=40, p=0.5),
    Cutout(mask_size=20, p=0.5),
    #transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Flip horizontal aléatoire
    transforms.RandomVerticalFlip(),  # Flip vertical aléatoire
    transforms.RandomRotation(30),  # Rotation aléatoire jusqu'à 30 degrés
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajustements aléatoires des couleurs
    #transforms.RandomResizedCrop(224),  # Recadrage aléatoire avec redimensionnement à 224x224
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2)),  # Transformation affine aléatoire
    transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Perspective aléatoire
    transforms.RandomGrayscale(p=0.1),  # Convertir en niveaux de gris aléatoirement
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),  # Flou gaussien aléatoire
  # Convertir en Tensor
   # Normalisation avec les valeurs moyennes et std de ImageNet
    
    
])

test_transform = transforms.Compose([
    #transforms.ToPILImage(),
     #transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean.unsqueeze(0), std.unsqueeze(0)),
    #transforms.Resize((224, 224)),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
from torch.utils.data import Subset

train_subset = Subset(dataset, train_indices)
val_subset = Subset(dataset, val_indices)
test_subset = Subset(dataset, test_indices)

# Créer les instances de TransformingSubset
train_dataset = TransformingSubset(train_subset, transform=train_transform)

valid_dataset = TransformingSubset(val_subset, transform=test_transform)
custum_test_dataset = TransformingSubset(test_subset, transform=test_transform)

In [ ]:
# from collections import Counter
# # Compter le nombre d'images par classe
# counter = Counter(train_dataset.labels)
# classes = list(counter.keys())
# counts = list(counter.values())

# # Visualiser la distribution des classes
# plt.figure(figsize=(10, 5))
# plt.bar(classes, counts, color='blue')
# plt.xlabel('Classe')
# plt.ylabel('Nombre d\'images')
# plt.title('Distribution des images par classe')
# plt.xticks(classes)
# plt.show()

In [ ]:
# train_transform

In [ ]:
# from collections import Counter
# # Compter le nombre d'images par classe
# counter = Counter(dataset_balance.balanced_labels)
# classes = list(counter.keys())
# counts = list(counter.values())

# # Visualiser la distribution des classes
# plt.figure(figsize=(10, 5))
# plt.bar(classes, counts, color='blue')
# plt.xlabel('Classe')
# plt.ylabel('Nombre d\'images')
# plt.title('Distribution des images par classe')
# plt.xticks(classes)
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
for x,y,z,n in train_dataset:
    print(y) # afficher le nom de l'image   
    print(z) # afficher le nom de l'image
    print(x) # afficher le nom de l'image

  # Afficher le spectrogramme
    #print(x.shape)
    plt.imshow(x.permute(1, 2, 0), cmap='gray')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Spectrogram Label: {dataset.index_to_label[z]}")
    plt.xlabel("Time")
    plt.ylabel("Frequency")
    plt.show()
    break

In [ ]:
len(train_dataset),len(valid_dataset),len(custum_test_dataset)